## Publish a Domino Model Endpoint

Prepare for Model Deployment. You can deploy the file `kafka_model.py`. Deploying the model requires the following:-
1. Environment Id - You can get a list of environments by invoking `list_envs` function below
2. Python file (`kafka_model.py` in our case)
3. Function in the Python file  (`predict` in out case)
4. Any additional environment variables the model needs to configure itself. We pass the following:
   - KAFKA_BOOTSTRAP_SERVERS = 'KAFKA_BOOTSTRAP_SERVERS'
   - KAFKA_USER_NAME = 'KAFKA_USER_NAME'
   - KAFKA_PASSWORD = 'KAFKA_PASSWORD'
   - MODEL_TOPIC_NAME_PREFIX = 'MODEL_TOPIC_NAME_PREFIX' (There are three topics with the same prefix)
   - INFERENCE_GROUP_ID = 'INFERENCE_GROUP_ID' (A model can have many instances to serve, each of them should use the same Consumer Group Id to ensure a Kafka message is usually processed only once)
   - PREDICTION_TOPIC_SUFFIX = A suffix the prediction topic




In [42]:
from domino import Domino
import os
import time
api_host = os.getenv("DOMINO_API_HOST")
project_id=os.environ['DOMINO_PROJECT_ID']
dm_api = Domino(
        "{}/{}".format(
            os.getenv("DOMINO_PROJECT_OWNER"), os.getenv("DOMINO_PROJECT_NAME")
        )
    )

def list_envs():
    all_available_environments = dm_api.environments_list()
    global_environments = list(
        filter(
            lambda x: x.get("visibility") == "Global",
            all_available_environments["data"],
        )
    )
    return all_available_environments["data"]

# nv_pairs is a dictionary of name/value pairs, {'name': 'value'}
def add_env_vars(model_id, nv_pairs):
    vars_array = [{"name": name, "value": value} for (name, value) in nv_pairs.items()]
    request = {"vars": vars_array}
    api_host = os.getenv("DOMINO_API_HOST")
    resp = dm_api.request_manager.post(
        f"{api_host}/models/{model_id}/environmentVariables/add", json=request
    )


Get a list of all environments

```
[{'id': '63fa71e8e05a501c43a61ac6', 'name': 'KAFKA', 'visibility': 'Private'},
 {'id': '63f7c9e768e90a0b79d68d16',
  'name': '5.5 Snowflake Model Export Environment (Domino Internal)',
  'visibility': 'Global'},
 {'id': '63f7c9e768e90a0b79d68d13',
  'name': '5.5 Spark Compute Environment',
  'visibility': 'Global'},
 {'id': '63f7c9e768e90a0b79d68d12',
  'name': '5.5 Spark Cluster Environment',
  'visibility': 'Global'},
 {'id': '63f7c9e768e90a0b79d68d11',
  'name': 'Domino Standard Environment Py3.9 R4.2',
  'visibility': 'Global'}]
```

In [43]:
list_envs()

[{'id': '641915929bc2112a585a2823',
  'name': 'icde-23',
  'visibility': 'Private'},
 {'id': '64190b169bc2112a585a27a8',
  'name': 'OnlineLearningEnv-v2',
  'visibility': 'Private'},
 {'id': '641747b89bc2112a585a2572',
  'name': 'OnlineLearningEnv',
  'visibility': 'Private'},
 {'id': '640fa4b378a49e32198d8f37',
  'name': 'WorkspaceSparkEMR-v2',
  'visibility': 'Private'},
 {'id': '640623f113c07a555966e698',
  'name': 'spark-template-env',
  'visibility': 'Private'},
 {'id': '63f4e44a62d9464871ffcac6',
  'name': 'WorkspaceSparkEMR',
  'visibility': 'Private'},
 {'id': '63e3ba168a2d5a33e515af09',
  'name': '5.4 Snowflake Model Export Environment (Domino Internal)',
  'visibility': 'Global'},
 {'id': '63e3ba168a2d5a33e515af08',
  'name': '5.4 Spark Compute Environment',
  'visibility': 'Global'},
 {'id': '63e3ba168a2d5a33e515af07',
  'name': '5.4 Spark Cluster Environment',
  'visibility': 'Global'},
 {'id': '63e3ba168a2d5a33e515af06',
  'name': 'Domino Standard Environment Py3.9 R4.2',


In [44]:

import os
env_id='641915929bc2112a585a2823'

model_file='src/online_learning_model.py'
model_function='predict'


KAFKA_BOOTSTRAP_SERVERS = 'kafka_bootstrap_servers'
KAFKA_USER_NAME = 'kafka_username'
KAFKA_PASSWORD = 'kafka_password'
FEATURE_TOPIC_NAME = 'FEATURE_TOPIC_NAME'
INFERENCE_GROUP_ID = 'INFERENCE_GROUP_ID'
PREDICTION_TOPIC_PREFIX='PREDICTION_TOPIC_PREFIX'
PREDICTION_TOPIC_SUFFIX='PREDICTION_TOPIC_SUFFIX'
model_name= os.environ['DOMINO_PROJECT_NAME'] 
env_variables = {
                 KAFKA_BOOTSTRAP_SERVERS:os.environ[KAFKA_BOOTSTRAP_SERVERS],
                 KAFKA_USER_NAME:os.environ[KAFKA_USER_NAME],
                 KAFKA_PASSWORD:os.environ[KAFKA_PASSWORD],
                 FEATURE_TOPIC_NAME: 'malicious_url_events',
                 INFERENCE_GROUP_ID: f'{model_name}-03-21-06PM',
                 PREDICTION_TOPIC_PREFIX: 'malicious_url_predictions',
                 PREDICTION_TOPIC_SUFFIX: '_16',
                }
 

## Publish Model

In [45]:
published_model = dm_api.model_publish(
        file=model_file,
        function=model_function,
        environment_id=env_id,
        name=f'{model_name}-v8',
        description="Autopublish of MLFLOW model {}".format(model_name),
    )
published_model_id = published_model.get("data", {}).get("_id")
print(f'Published Model{published_model_id}')
print('Now add env variables')

add_env_vars(published_model_id,env_variables)
resp = dm_api.request_manager.get(f'{api_host}/models/{published_model_id}/activeStatus',json={})

status = ''
if resp.status_code==200:
    status = resp.json()['status']
print(status)
resp = dm_api.request_manager.get(
        f"{api_host}/models/{published_model_id}/activeStatus", json={}
    )
while resp.json()['isPending']:
    print('Sleeping for 30 seconds')
    time.sleep(30)
    
    resp = dm_api.request_manager.get(
        f"{api_host}/models/{published_model_id}/activeStatus", json={}
    )
    print(resp.json())
status = resp.json()['status']
print(status)

Published Model641a1cf99bc2112a585a2911
Now add env variables
Stopped
Sleeping for 30 seconds
{'modelId': {'value': '641a1cf99bc2112a585a2911'}, 'modelVersionId': {'value': '641a1cf99bc2112a585a2913'}, 'status': 'Ready to run', 'operations': [{'startTime': 1679432953497, 'lastUpdated': 1679432954095, 'sagaDescription': 'Deploy model version', 'shortStateDescription': 'Building', 'longStateDescription': 'Image Building', 'isFailure': False}], 'lastOperation': {'startTime': 1679432953497, 'lastUpdated': 1679432954095, 'sagaDescription': 'Deploy model version', 'shortStateDescription': 'Building', 'longStateDescription': 'Image Building', 'isFailure': False}, 'isPending': True}
Sleeping for 30 seconds
{'modelId': {'value': '641a1cf99bc2112a585a2911'}, 'modelVersionId': {'value': '641a1cf99bc2112a585a2913'}, 'status': 'Ready to run', 'operations': [{'startTime': 1679432953497, 'lastUpdated': 1679432954095, 'sagaDescription': 'Deploy model version', 'shortStateDescription': 'Building', 'lon

In [ ]:
import time

In [ ]:
time.sleep(1)

In [33]:
import online_learning_model

KeyError: 'FEATURE_TOPIC_NAME'